# Cuaderno 13: Problemas de caminos más cortos 
# (shortest path problems)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido $D=(V,A)$; 
* un vector de costos $c \in \ZZ^{A}$  asociados a los arcos de $D$; 
* un nodo de salida $s$; y,
* un nodo de llegada $t$.

El *problema del camino más corto (shortest path problem, SPP)* consiste en encontrar un camino desde $s$ hasta $t$ cuyo costo sea mínimo. El costo de un camino se calcula sumando de los costos de sus arcos.

Suponer que $D$ no contiene circuitos de costo negativo. Utilizando variables binarias $x_{ij}$ para indicar si los arcos de $A$ son seleccionados o no para formar parte del camino, este problema puede formularse como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(s, i) \in A} x_{si} = 1,\\
&\sum_{(i, t) \in A} x_{it} = 1,\\
& \sum_{(j, i) \in A} x_{ji} - \sum_{(i, j) \in A} x_{ij} = 0, \quad \forall i \in V \setminus \tabulatedset{s,t},\\
& x_{ij} \in \tabulatedset{0, 1}, \quad \forall (i, j) \in A.
\end{align*}

La función objetivo mide la suma de los costos de los arcos seleccionados.

La primera restricción especifica que debe seleccionarse exactamente un arco saliente del nodo $s$. 

De manera similar, la segunda restricción indica que debe seleccionarse exactamente un arco entrante a $t$.

Por último, la tercera familia de restricciones requiere que la cantidad de arcos entrantes a un nodo $i \in V \setminus \tabulatedset{s,t}$ que sean seleccionados dentro de la solución debe coincidir con la cantidad de arcos salientes del mismo nodo.

Notar que todo camino de $s$ a $t$ satisface las tres restricciones del modelo. Por otra parte, existen otras clases de grafos que satisfacen estas restricciones. (Pensar en algún ejemplo.) Sin embargo, puede demostrarse que si $D$ no contiene *circuitos de costo negativo* entonces la solución óptima del modelo corresponde al camino de costo mínimo de $s$ a $t$.

Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los conjuntos y parámetros del modelo:

In [ ]:
from gurobipy import *

# Nodos del grafo
V = tuplelist(range(1,7))

# Arcos y sus costos 
A, c = multidict({
  (1, 2):  3,
  (1, 3):  20,
  (2, 3):  10,
  (2, 4):  5,
  (4, 3):  2,
  (3, 5):  2,
  (4, 5):  5,
  (4, 6):  5,
  (5, 6):  2})

# Nodo de salida
s = 1

# Nodo de llegada
t = 6

# --- los valores a partir de aqui se calculan automaticamente ---
# nodos internos: Vi := V \ {s, t}
Vi = tuplelist([i for i in V if i!=s and i!=t])

La siguiente celda utiliza los módulos `NetworkX` y `matplotlib` para graficar esta instancia:

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
D = nx.DiGraph()
D.add_nodes_from(V)
node_labels= {i : str(i) for i in V}
D.add_edges_from(A)
edge_labels = {(i,j) : str(c[i,j]) for (i,j) in A}
plt.figure(figsize=(10,5))
pos = {1 : (1,2), 2 : (2,3), 3 : (2,1), 4 : (3,3), 5 : (3,1), 6:(4,2)}
nx.draw_networkx_edge_labels(D, pos, edge_labels)
nx.draw_networkx(D, pos, labels= node_labels, node_color='cyan', node_size=500)


Creamos ahora el objeto modelo y las variables de binarias de selección de arcos $x_{ij}$. Observar que las variables están indexadas por los conjuntos de arcos.

In [ ]:
# Crear el objeto modelo
m = Model('shortest-path')

# Crear las variables
x = m.addVars(A, name="x", vtype=GRB.BINARY)

Definimos la función objetivo a minimizar:

In [ ]:
m.setObjective(x.prod(c, '*'), GRB.MINIMIZE)

Definimos las restricciones del modelo:

In [ ]:
# Arcos salientes de s
m.addConstr(x.sum(s,'*')  == 1, "grado_s")

# Arcos entrantes a t
m.addConstr(x.sum('*', t)  == 1, "grado_t")

# Balance de grados en los demas nodos
m.addConstrs(
    (x.sum('*',i) - x.sum(i,'*')  == 0 for i in Vi), "grado")

Finalmente, resolvemos el modelo y mostramos la solución. Notar que los valores de las variables pueden recuperarse también empleando el método `getAttr`:

In [ ]:
# Calcular la solucion optima
m.optimize()

# Escribir la solucion
if m.status == GRB.Status.OPTIMAL:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('Arcos seleccionados:')
    for i,j in A:
        if vx[i,j] >= 0.99:
            print('{} -> {}'.format(i, j))

En la siguiente celda graficamos la solución empleando `networkx` y `ipycytoscape`:

In [ ]:
import networkx as nx
import ipycytoscape
D = nx.DiGraph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['etiq']= str(i)
D.add_edges_from(A)
for i,j in A:
    D.edges[i,j]['etiq'] = str(c[i,j])
    D.edges[i,j]['color'] =  '#9dbaea' if vx[i,j]<=0.1 else '#ff007f'
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=True)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 4, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementacion de modelos de programacion lineal entera
# Problema de caminos mas cortos (shortest path problem, SPP)
# Luis M. Torres (EPN 2021)

from gurobipy import *

# Nodos del grafo
V = tuplelist(range(1,7))

# Arcos y sus costos 
A, c = multidict({
  (1, 2):  3,
  (1, 3):  20,
  (2, 3):  10,
  (2, 4):  5,
  (4, 3):  2,
  (3, 5):  2,
  (4, 5):  5,
  (4, 6):  10,
  (5, 6):  2})

# Nodo de salida
s = 1

# Nodo de llegada
t = 6

# --- los valores a partir de aqui se calculan automaticamente ---
# nodos internos: Vi := V \ {s, t}
Vi = tuplelist([i for i in V if i!=s and i!=t])

try:
    # Crear el objeto modelo
    m = Model('shortest-path')

    # Crear las variables
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # Definir la funcion objetivo
    m.setObjective(x.prod(c, '*'), GRB.MINIMIZE)

    # Arcos salientes de s
    m.addConstr(x.sum(s,'*')  == 1, "grado_s")

    # Arcos entrantes a t
    m.addConstr(x.sum('*', t)  == 1, "grado_t")

    # Balance de grados en los demas nodos
    m.addConstrs(
        (x.sum('*',i) - x.sum(i,'*')  == 0 for i in Vi), "grado")

    # Calcular la solucion optima
    m.optimize()

    # Escribir la solucion
    if m.status == GRB.Status.OPTIMAL:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('Arcos seleccionados:')
        for i,j in A:
            if vx[i,j] > 0:
                print('{} -> {}'.format(i, j))
            
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

In [ ]:
m.write('caminos.lp')